In [1]:
import pandas as pd
bcsc_paths = [
    "../data/bcsc_risk_factors_summarized1_092020.csv",
    "../data/bcsc_risk_factors_summarized2_092020.csv",
    "../data/bcsc_risk_factors_summarized3_092020.csv",
]
df_list = []
for path in bcsc_paths:
    bcsc = pd.read_csv(path)
    df_list.append(bcsc)
bcsc_df = pd.concat(df_list, ignore_index=True)

bcsc_df_clean = (bcsc_df[~(bcsc_df.iloc[:, 2:12] == 9).any(axis=1)].reset_index(drop=True))
bcsc_df_clean.to_csv("../data/bcsc_concatenated_no_hist9.csv", index=False)
print(f"saved into the data folder")

saved into the data folder


In [2]:
!pip install xgboost


In [3]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, roc_auc_score
import xgboost as xgb
import joblib

df = pd.read_csv("../data/bcsc_concatenated_no_hist9.csv")
##Separate features (X) and target (y)
X = df.drop(columns=["breast_cancer_history"])
y = df["breast_cancer_history"]


# 9) Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y if len(np.unique(y))>1 else None
)
print(f"Training shape: {X_train.shape}, Test shape: {X_test.shape}")

# 10) Train XGBoost classifier
model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric="logloss",
    random_state=42
)
model.fit(X_train, y_train)
print("Model training finished.")

# 11) Evaluate
y_pred = model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))
try:
    y_pred_prob = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred_prob)
    print(f"ROC AUC: {auc:.4f}")
except:
    print("Could not compute ROC AUC (possible single-class).")

# 12) Ensure data directory exists before saving
os.makedirs(DATA_DIR, exist_ok=True)

# 13) Save artifacts
MODEL_PATH = os.path.join(DATA_DIR, "bcsc_xgb_model.pkl")
ENCODERS_PATH = os.path.join(DATA_DIR, "bcsc_feature_encoders.pkl")
joblib.dump(model, MODEL_PATH)
print(f"Saved model to {MODEL_PATH}")
joblib.dump(feature_encoders, ENCODERS_PATH)
print(f"Saved feature encoders to {ENCODERS_PATH}")

if target_le is not None:
    TARGET_PATH = os.path.join(DATA_DIR, "bcsc_target_encoder.pkl")
    joblib.dump(target_le, TARGET_PATH)
    print(f"Saved target encoder to {TARGET_PATH}")


Training shape: (276051, 12), Test shape: (69013, 12)


C:\Users\vetdd\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:02:27] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Model training finished.
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.98      0.93     58427
           1       0.67      0.26      0.38     10586

    accuracy                           0.87     69013
   macro avg       0.78      0.62      0.65     69013
weighted avg       0.85      0.87      0.84     69013

ROC AUC: 0.8843


NameError: name 'DATA_DIR' is not defined

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, roc_auc_score
import xgboost as xgb
import joblib

# 1) Load data
DATA_DIR = "data"
df = pd.read_csv(os.path.join(DATA_DIR, "bcsc_concatenated_no_hist9.csv"))
X = df.drop(columns="breast_cancer_history")
y = df["breast_cancer_history"]

# 2) Train/valid/test split
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=42
)
X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=42
)

# 3) Compute pos‐weight
neg, pos = np.bincount(y_train)
scale_pos_weight = neg / pos

# 4) Set up classifier & hyperparam search (no early-stopping here!)
base_clf = xgb.XGBClassifier(
    objective="binary:logistic",
    use_label_encoder=False,
    eval_metric="auc",
    scale_pos_weight=scale_pos_weight,
    random_state=42,
)

param_dist = {
    "n_estimators":    [100, 300, 500],
    "max_depth":       [3, 5, 7],
    "learning_rate":   [0.01, 0.05, 0.1],
    "subsample":       [0.6, 0.8, 1.0],
    "colsample_bytree":[0.6, 0.8, 1.0],
    "gamma":           [0, 1, 5],
}

search = RandomizedSearchCV(
    estimator=base_clf,
    param_distributions=param_dist,
    n_iter=20,
    scoring="roc_auc",
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# 5) Fit search on TRAIN only
search.fit(X_train, y_train)
print("Best hyperparameters:", search.best_params_)

# 6) Take best estimator and refit with early stopping on train+valid
best = search.best_estimator_
best.set_params(use_label_encoder=None, n_estimators=10000)
best.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid)],
    eval_metric="auc",
    callbacks=[
        xgb.callback.EarlyStopping(rounds=25, save_best=True)],
    verbose=False
)

# 7) Evaluate on TEST
y_pred      = best.predict(X_test)
y_pred_prob = best.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print("Test ROC AUC:", roc_auc_score(y_test, y_pred_prob))

# 8) Save
os.makedirs(DATA_DIR, exist_ok=True)
joblib.dump(best, os.path.join(DATA_DIR, "bcsc_xgb_model.pkl"))
print("Saved model to", os.path.join(DATA_DIR, "bcsc_xgb_model.pkl"))
